In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import json
import scipy.io as sio
import cv2
import imageio
import matplotlib.gridspec as gridspec

In [ ]:
%run GetLossObj.ipynb

In [ ]:
models = {}
# modelNumbers = [156, 161, 159, 158, 162, 160, 155, 154]
modelNumbers = [172, 173]
baseDir = "../save/models/model{}/"
for modelNumber in modelNumbers:
    models[modelNumber] = {}
    dirs = baseDir.format(modelNumber)
    with open(dirs + "args.txt") as f:
        args = f.read()
    args = Bunch(json.loads(args))
    args.no_cuda = True
    args.cuda = False
    args.dropout= 0.0
    if args.dataset == "traffic":
        args.output_dim = args.x_dim
    else: #args.dataset == "human":
        args.output_dim = args.x_dim * 2
    models[modelNumber]["args"] = args
    model = RecurrentSeq2Seq(
                args.h_dim,
                args.dropout_prob,
                args.rnn_type,
                args.bidirectionalEncoder,
                args.n_layers,
                args.attention_type,
                args.input_feeding,
                args.x_dim * args.channels,
                args.output_dim,
                args)
    desired_state_dict = torch.load(dirs+"Seq2SeqAttn_full_model.pth", map_location=lambda storage, loc: storage)
    model.load_state_dict(desired_state_dict)
    model.eval()
    
    models[modelNumber]["model"] = model


In [ ]:
def show2D(img, predPoints, targetPoints):
    ref = {"nJoints" : 16,
           "accIdxs" : [0, 1, 2, 3, 4, 5, 10, 11, 14, 15],
           "shuffleRef" : [[0, 5], [1, 4], [2, 3], 
                 [10, 15], [11, 14], [12, 13]],
           "edges" : [[0, 1], [1, 2], [2, 6], [6, 3], [3, 4], [4, 5], 
             [10, 11], [11, 12], [12, 8], [8, 13], [13, 14], [14, 15], 
             [6, 8], [8, 9]]
          }
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.33
    fontColor = (255,255,255)
    lineType = 2
    for points, c, of in zip([predPoints, targetPoints], [(255,0,0), (0,0,255)], [0,100]):
        points = ((points.reshape(ref["nJoints"], -1))).astype(np.int32)
        for e in ref["edges"]:
            cv2.line(img, (points[e[0], 0]+of, points[e[0], 1]), (points[e[1], 0]+of, points[e[1], 1]), (0,0,0), 2)
        for j in range(ref["nJoints"]):
            #cv2.putText(img, str(j), (points[j, 0], points[j, 1]), font, fontScale, fontColor, lineType)
            cv2.circle(img, (points[j, 0]+of, points[j, 1]), 5, c, -1)
    return img

In [ ]:
dataDir = "/home/dan/data/Human/Processed/INPUT_HORIZON_25_PREDICTION_HORIZON_50/"
f = h5py.File(os.path.join(dataDir, "test.h5"), "r")

In [ ]:
_, scaler, _, _, _, _ = getHumanDataset(dataDir, "test")

In [ ]:
inputTimes = np.arange(1, 25 * 5 + 1, 5)
targetTimes = np.arange(1 + 25*5, 1 + 75*5, 5)

In [ ]:
f["input2d"].shape[0] / 64

In [ ]:
allTargets = []
allInputs = []
nBatches = int(np.floor(f["input2d"].shape[0] / 64))
for model in models.values():    
    model["allOutputs"] = []
loss = nn.L1Loss()
for i, batchStop in enumerate(range(64, 64 * (nBatches + 1), 64)):
    print(i + 1," / ",nBatches)
    inputT = torch.Tensor(f["input2d"][batchStop - 64:batchStop])
    targetT = torch.Tensor(f["target2d"][batchStop - 64:batchStop])
    transInput, transTarget = scaler.transformBatchForEpoch((inputT, targetT))
    allTargets.append(targetT.cpu().detach().numpy())
    allInputs.append(inputT.cpu().detach().numpy())
    for model in models.values():
        output = model["model"](transInput, transTarget, 0)
        reconOutput = scaler.inverse_transform(output)
        model["allOutputs"].append(reconOutput.cpu().detach().numpy())


In [ ]:
for model in models.values():
    model["allOutputs"] = np.concatenate(model["allOutputs"], axis=0)
allInputs = np.concatenate(allInputs, axis=0)
allTargets = np.concatenate(allTargets, axis=0)

In [ ]:
for model in models.values():
    model["allOutputs"] = np.transpose(model["allOutputs"], (0,1,3,2))

In [ ]:
allInputs.shape

In [ ]:
models[modelNumbers[0]]["allOutputs"].shape

In [ ]:
allTargets.shape

In [ ]:
for modelNum, model in models.items():
    losses = []
    for stopSec in [10,20,30,40,50]:
        losses.append(np.mean(np.abs(model["allOutputs"][:,:stopSec,:,:] - allTargets[:,:stopSec,:,:])))
    print(modelNum)
    print("{:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f}".format(*losses))

In [ ]:
for _ in range(10):
    inst = np.random.choice(models[modelNumbers[0]]["allOutputs"].shape[0])
    def getImg(allOutputs, timeStepIdx):
        whiteImage = cv2.imread("/Users/danielzeiberg/Downloads/white.png")
        return show2D(whiteImage, allOutputs[inst,timeStepIdx], allTargets[inst,timeStepIdx])

    for modelNum in modelNumbers:
        modelDir = baseDir.format(modelNum)
        predNum = 0
        while os.path.isfile(modelDir+"motionPred{}Test.png".format(predNum)):
            predNum += 1
        images = []
        for i in range(50):
            whtIMG = cv2.imread("/home/dan/white.png")
            img = show2D(whtIMG, models[modelNum]["allOutputs"][inst, i], allTargets[inst, i])
            images.append(img)
        imageio.mimsave(modelDir+"motionPred{}Test.gif".format(predNum), images, fps=4)
        plt.figure(figsize = (8,12))
    #     gs1 = gridspec.GridSpec(10, 5, wspace=0, hspace=0)
    #     plt.figure(figsize = (6,6)) # set the figure size to be square
        gs1 = gridspec.GridSpec(10, 5)
        # set the space between subplots and the position of the subplots in the figure
        gs1.update(wspace=0.0, hspace=0.0, left = 0.0, right = 5.0, bottom = 0.0, top = 5.0)
    #     gs1.update(wspace=0.0001, hspace=0.0001) # set the spacing between axes. 
        for i in range(50):
           # i = i + 1 # grid spec indexes from 0
            ax1 = plt.subplot(gs1[i])
            plt.axis('on')
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
            ax1.set_aspect('equal')
    #         ax1.plot([0,1],[0,0])
            ax1.imshow(images[i-1].astype(np.uint8))
        plt.savefig(modelDir+"motionPred{}Test.png".format(predNum), bbox_inches='tight')

In [ ]:
np.unique(f["subject"][:6336]), np.unique(f["action"][:6336]), np.unique(f["subaction"][:6336]), np.unique(f["camera"][:6336])

In [ ]:
whtIMG = cv2.imread("/home/dan/white.png")
img = show2D(whtIMG, models[modelNumbers[0]]["allOutputs"][0,0], allTargets[0,0])

In [ ]:
type(img)

In [ ]:
plt.imshow(img)

In [ ]:
plt.figure(figsize = (10,10))
gs1 = gridspec.GridSpec(4, 4)
gs1.update(wspace=0.025, hspace=0.05) # set the spacing between axes. 

for i in range(16):
   # i = i + 1 # grid spec indexes from 0
    ax1 = plt.subplot(gs1[i])
    plt.axis('off')
    ax1.set_xticklabels([])
    ax1.set_yticklabels([])
    ax1.set_aspect('equal')
    ax1.plot([0,1],[0,1])
#     plt.subp

plt.show()